In [ ]:
from glob import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import segmentation

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set(font_scale = 2)

# Generar el DataFrame con las direcciones de las imagenes y sus respectivas mascaras.

In [ ]:
file_images = glob('../input/nerveutp/UTP-Nerve/*.png')
file_images.sort()
filepath_image = [] # sólo imagenes
filepath_mask = [] # mascaras
nerve_name = []
for filepath in [filepath_ for filepath_ in file_images if 'mask' not in filepath_]:
  mask = filepath[:-4]+'_mask.png'
  if mask in file_images:
    filepath_image.append(filepath)
    filepath_mask.append(mask)

    if 'ciatico' in filepath:
      nerve_name.append('ciatico')
    elif 'cubital' in filepath:
      nerve_name.append('cubital')
    elif 'femoral' in filepath:
      nerve_name.append('femoral')
    elif 'mediano' in filepath:
      nerve_name.append('mediano')

df = pd.DataFrame({'filepath':filepath_image,'nerve_name':nerve_name,'mask':filepath_mask})
t = df['nerve_name']
df.head()

# Fragmentar los conjuntos de Entrenamiento, Validación y Test

In [ ]:
seed = 19931218
df_train_images,df_test_images,t_train,_ = train_test_split(df,t, test_size=0.2,stratify = t,random_state=seed)
df_train_images,df_val_images = train_test_split(df_train_images, test_size=0.2,stratify = t_train,random_state=seed)

In [ ]:
img = cv2.imread(filepath_image[0])
mask = cv2.imread(filepath_mask[0])
height,width,_ = img.shape
print(height,'x',width)
plt.subplot(1,2,1)
plt.imshow(img)
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(mask*255,cmap='gray')
plt.axis('off')
plt.show()

# Creación de los generadores de datos

In [ ]:
"""
Se generan generadores tanto para las imágenes de Ultrasonido como para las mascaras. Luego, estas se compactan
en uno sólo por medio de la función ZIP, para generar un hgenerador de imagen y su mascara. 

"""
batch_size = 32
height,width = 256,256
image_datagen = ImageDataGenerator(rescale=1./255)

image_datagen_mask = ImageDataGenerator(rescale=1.0)

image_val_datagen = ImageDataGenerator(rescale = 1./255)
image_val_datagen_mask = ImageDataGenerator(rescale = 1.0)

generator_train_img = image_datagen.flow_from_dataframe(df_train_images,
                                                        x_col = 'filepath',
                                                        batch_size = batch_size,
                                                        class_mode = None,
                                                        directory = None,
                                                        target_size = (height,width),
                                                        seed = seed)

generator_train_mask = image_datagen_mask.flow_from_dataframe(df_train_images,
                                                         x_col='mask',
                                                         class_mode = None,
                                                         directory = None,
                                                         color_mode="grayscale", 
                                                         batch_size = batch_size,
                                                         target_size = (height,width),
                                                         seed = seed)

train_gen = zip(generator_train_img,generator_train_mask)


generator_val_img = image_val_datagen.flow_from_dataframe(df_val_images,
                                                        x_col='filepath',
                                                        class_mode = None,
                                                        directory = None,
                                                        target_size = (height,width),
                                                        batch_size = batch_size,
                                                        seed = seed)

generator_val_mask = image_val_datagen_mask.flow_from_dataframe(df_val_images,
                                                         x_col='mask',
                                                         class_mode = None,
                                                         directory = None,
                                                         color_mode="grayscale",
                                                         target_size = (height,width),
                                                         batch_size = batch_size,
                                                         seed = seed)

val_gen = zip(generator_val_img,generator_val_mask)


generator_test_img = image_val_datagen.flow_from_dataframe(df_test_images,
                                                        x_col='filepath',
                                                        class_mode = None,
                                                        directory = None,
                                                        target_size = (height,width),
                                                        batch_size = batch_size,
                                                        seed = seed)

generator_test_mask = image_val_datagen_mask.flow_from_dataframe(df_test_images,
                                                         x_col='mask',
                                                         class_mode = None,
                                                         directory = None,
                                                         color_mode="grayscale",
                                                         target_size = (height,width),
                                                         batch_size = batch_size,
                                                         seed = seed)

test_gen = zip(generator_test_img,generator_test_mask)


In [ ]:
i,j = next(train_gen)

plt.imshow(i[0,:,:,:])
edges_est = segmentation.clear_border(j[0,:,:,0])
plt.contour(edges_est,[0.5],colors=['red'])
plt.axis('off')
plt.show()

# Creación y entrenamiento de modelo